<a href="https://colab.research.google.com/github/DaeSeokSong/image-processing/blob/feature%2FUnet-scar/Unet_Scar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ※ Precautions

1.   RawDataset_Processor 실행 후 UNet 실행 추가




# 『Reference』

* *Paper*
>   [U-net](https://paperswithcode.com/paper/u-net-convolutional-networks-for-biomedical)
>
> 기존 CNN은 Single classification task에 사용되었지만,
> 
> biomedical image processing 분야에서는 한 이미지 내의 모든 pixel을 classification 하는 Semantic segmentation task가 중요하게 사용되었다.
>
>  sliding window 방식을 사용하는 CNN 구조와 달리 검증된 patch는 넘기기 때문에 보다 빠른 처리가 가능한 구조이다.
> 
> 적은 양의 데이터로도 dataset argumentation을 통해 잘 학습시킬 수 있다.
>
>   * [U-net++](https://paperswithcode.com/paper/unet-a-nested-u-net-architecture-for-medical)
>   * [ResUNet++](https://paperswithcode.com/paper/resunet-an-advanced-architecture-for-medical)

<br>

* *Lecture*
> * [UNet architecture by pytorch](https://89douner.tistory.com/300)



# **1.Development enviroment**

## *1) Import*

### 1-1) Library

In [ ]:
# U-net
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

from torchvision import transforms

# Image processing
import cv2
import numpy as np
import pickle as pl
import matplotlib.pyplot as plt

from google.colab.patches import cv2_imshow
from google.colab import output

from PIL import Image

# ETC
import os
import time
import math

### 1-2) Mount google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd /content/gdrive/MyDrive/Models/GAN_Scar
!ls -al

/content/gdrive/MyDrive/Models/GAN_Scar
total 188
drwx------ 4 root root  4096 Aug 16 08:53  Dataset
-rw------- 1 root root 42616 Aug 30 13:26  Image_segmentation-Scar.ipynb
drwx------ 4 root root  4096 Aug 31 14:31  Log
-rw------- 1 root root 34986 Aug 29 15:15  Processor_PerformanceTester-Scar.ipynb
drwx------ 2 root root  4096 Aug 16 08:39  Raw_Dataset
-rw------- 1 root root 16647 Aug 29 15:29  RawDataset_Processor-Scar.ipynb
drwx------ 2 root root  4096 Aug 23 14:47  result
-rw------- 1 root root 39995 Aug 15 11:40 'UNet architecture.PNG'
-rw------- 1 root root 40422 Aug 31 14:33  Unet-Scar.ipynb


# **2.Train U-Net**

## *1) Grobal variable*

In [ ]:
# Path
MODEL_PATH = "/content/gdrive/MyDrive/Models/GAN_Scar"

DATASET_PATH = "/Dataset"
LOG_PATH = "/Log"
CHECK_POINT_PATH = '/CheckPoint'

TRAIN_PATH = "/train"
VAL_PATH = "/val"
TEST_PATH = "/test"

# Train hyperparameter
LR = 1e-3
BATCH_SIZE = 8 # batch size 8 초과부터는 Colab gpu ram 용량 초과로 원활한 학습 불가
EPOCHS = 20

"""
GPU 사용이 가능하면 cuda 사용
아니면 CPU를 이용하여 학습
"""
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## *2) Funtion*

### 2-1) Save learned model (구현중)

In [ ]:
# Save network
def save(ckpt_dir, net, opt, epoch):
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)

    torch.save({'net':net.state_dict(), 'opt':opt.state_dict()}, '%s/model_epoch%d.pth'%(ckpt_dir,epoch))

# Load network
def load(ckpt_dir, net, opt):
    if not os.path.exists(ckpt_dir): # 저장된 네트워크가 없다면 인풋을 그대로 반환
        epoch = 0

        return net, opt, epoch
    
    ckpt_lst = os.listdir(ckpt_dir) # ckpt_dir 아래 있는 모든 파일 리스트를 받아온다
    ckpt_lst.sort(key = lambda f : int(''.join(filter(str, isdigit, f))))

    dict_model = torch.load('%s/%s' % (ckpt_dir, ckpt_lst[-1]))

    net.load_state_dict(dict_model['net'])
    opt.load_state_dict(dict_model['opt'])
    epoch = int(ckpt_lst[-1].split('epoch')[1].split('.pth')[0])

    return net, opt, epoch

### 2-2) Convenience func

In [ ]:
def imshow_waitkey_enter(image):
    cv2_imshow(image)

    time.sleep(0.5)
    
    input("Please press the Enter key to proceed\n")
    output.clear()

    pass

## *3) Class*

### 3-1) Custom U-Net

#### 3-1-1) Architecture

**마지막 Output 채널 2 → 1 변경**
> Binary classification

<img src = "https://drive.google.com/uc?id=14CzAAaKv5v7pVfvugBRbD1xI4IuhmoyT"  width = 640>

#### 3-1-2) Build network

In [ ]:
# torch.nn의 Module 클래스를 상속한, 커스텀 UNet 클래스
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()

        # kernel size, stride, padding, bias는 거의 고정 >> predefine
        def ConvBatchReLU_2d(in_ch, out_ch, k_size=3, stride=1, padding=0, bias=True):
            layers = []

            # Add Conv layer
            layers += [nn.Conv2d(in_channels=in_ch,
                                 out_channels=out_ch,
                                 kernel_size=k_size,
                                 stride=stride,
                                 padding=padding,
                                 bias=bias
                                 )]

            # Add batch normalization layer
            layers += [nn.BatchNorm2d(num_features=out_ch)]

            # Add ReLU
            layers += [nn.ReLU()]

            # Define conv, ReLU step in contracting path
            CBR = nn.Sequential(*layers)

            return CBR

        """
        [Contracting path]
        >> 입력 이미지의 context 포착이 목적
        """
        # enc == encoder / n_m == n번째 stage(step)의 m번째 레이어
        self.enc1_1 = ConvBatchReLU_2d(in_ch=1, out_ch=64)
        self.enc1_2 = ConvBatchReLU_2d(in_ch=64, out_ch=64)

        self.pool1 = nn.MaxPool2d(kernel_size=2)

        self.enc2_1 = ConvBatchReLU_2d(in_ch=64, out_ch=128)
        self.enc2_2 = ConvBatchReLU_2d(in_ch=128, out_ch=128)

        self.pool2 = nn.MaxPool2d(kernel_size=2)

        self.enc3_1 = ConvBatchReLU_2d(in_ch=128, out_ch=256)
        self.enc3_2 = ConvBatchReLU_2d(in_ch=256, out_ch=256)

        self.pool3 = nn.MaxPool2d(kernel_size=2)

        self.enc4_1 = ConvBatchReLU_2d(in_ch=256, out_ch=512)
        self.enc4_2 = ConvBatchReLU_2d(in_ch=512, out_ch=512)

        self.pool4 = nn.MaxPool2d(kernel_size=2)

        self.enc5_1 = ConvBatchReLU_2d(in_ch=512, out_ch=1024)

        """
        [Expansive path]
        >> 세밀한 Localization을 위한 높은 차원의 채널을 갖는 Upsampling
        >> 얕은 레이어의 특집 맵을 결합
        """
        # dec == decoder
        self.dec5_1 = ConvBatchReLU_2d(in_ch=1024, out_ch=512)

        # up-conv 레이어는 채널을 복원을 해야하기 때문에 kernel size를
        # 대칭되는 MaxPool layer의 kernel size와 같도록 설정한다.
        self.unpool4 = nn.ConvTranspose2d(in_channels=512,
                                          out_channels=512,
                                          kernel_size=2,
                                          stride=2,
                                          padding=0,
                                          bias=True)
        
        # input channel은 up-conv와 대칭되는 enc, 두 레이어에서
        # 같은 크기의 채널로 오기 때문에 대칭 enc 레이어보다 input이 두 배 많다.
        self.dec4_2 = ConvBatchReLU_2d(in_ch=2 * 512, out_ch=512)
        self.dec4_1 = ConvBatchReLU_2d(in_ch=512, out_ch=256)

        self.unpool3 = nn.ConvTranspose2d(in_channels=256,
                                          out_channels=256,
                                          kernel_size=2,
                                          stride=2,
                                          padding=0,
                                          bias=True)
        
        self.dec3_2 = ConvBatchReLU_2d(in_ch=2 * 256, out_ch=256)
        self.dec3_1 = ConvBatchReLU_2d(in_ch=256, out_ch=128)

        self.unpool2 = nn.ConvTranspose2d(in_channels=128,
                                          out_channels=128,
                                          kernel_size=2,
                                          stride=2,
                                          padding=0,
                                          bias=True)
        
        self.dec2_2 = ConvBatchReLU_2d(in_ch=2 * 128, out_ch=128)
        self.dec2_1 = ConvBatchReLU_2d(in_ch=128, out_ch=64)

        self.unpool1 = nn.ConvTranspose2d(in_channels=64,
                                          out_channels=64,
                                          kernel_size=2,
                                          stride=2,
                                          padding=0,
                                          bias=True)
        
        self.dec1_2 = ConvBatchReLU_2d(in_ch=2 * 64, out_ch=64)
        self.dec1_1 = ConvBatchReLU_2d(in_ch=64, out_ch=64)

        # conv 1*1, N class for segmentation
        # 이미지 상에서는 out_channels 2라 되어있으나 결과 도출을 위해 1로 설정
        self.conv = nn.Conv2d(in_channels=64,
                              out_channels=1,
                              kernel_size=1,
                              stride=1,
                              padding=0,
                              bias=True)

    """
        [Skip connection]
        >> Semantic segmentation에서는 위치정보가 중요하기에
        >> 이에 대한 소실 방지 차원에서 이전 연산했던 값을 더해준다.

        >> copy and crop
        >> encoding 데이터가 더 크므로 복사(copy)후 잘라준다(crop)
    """
    def copy_and_crop(self, enc, unpool):
        cpy_enc =  enc.clone().cpu().detach().numpy()
        diff_y = cpy_enc.shape[2] - unpool.shape[2] # height
        diff_x = cpy_enc.shape[3] - unpool.shape[3] # width

        top = int(diff_y / 2)
        left = int(diff_x / 2)
        height = int(cpy_enc.shape[2] - top)
        width = int(cpy_enc.shape[3] - left)

        if height - top > unpool.shape[2]:
            height = height - ((height - top) - unpool.shape[2])
        elif height - top < unpool.shape[2]:
            height = height + (unpool.shape[2] - (height - top))

        if width - left > unpool.shape[3]:
            width = width - ((width - left) - unpool.shape[3])
        elif width - left < unpool.shape[3]:
            width = width + (unpool.shape[3] - (width - left))

        cpy_enc = cpy_enc[:, :, top:height, left:width]
        cpy_enc = torch.Tensor(cpy_enc).to(DEVICE)

        #print("encoding size = ", cpy_enc.shape)
        #print("unpooling size = ", unpool.shape)

        return torch.cat((unpool, cpy_enc), dim=1)

    # x == input_image
    def forward(self, x):
        #print("Input size = ", x.shape)
        input_width, input_height = x.shape[2], x.shape[3]

        # encoder part
        enc1_1 = self.enc1_1(x)
        enc1_2 = self.enc1_2(enc1_1)
        pool1 = self.pool1(enc1_2)
        #print("enc1_1 size = ", enc1_1.shape)
        #print("enc1_2 size = ", enc1_2.shape)
        #print("pool1 size = ", pool1.shape)

        enc2_1 = self.enc2_1(pool1)
        enc2_2 = self.enc2_2(enc2_1)
        pool2 = self.pool2(enc2_2)
        #print("enc2_1 size = ", enc2_1.shape)
        #print("enc2_2 size = ", enc2_2.shape)
        #print("pool2 size = ", pool2.shape)
        
        enc3_1 = self.enc3_1(pool2)
        enc3_2 = self.enc3_2(enc3_1)
        pool3 = self.pool3(enc3_2)
        #print("enc3_1 size = ", enc3_1.shape)
        #print("enc3_2 size = ", enc3_2.shape)
        #print("pool3 size = ", pool3.shape)

        enc4_1 = self.enc4_1(pool3)
        enc4_2 = self.enc4_2(enc4_1)
        pool4 = self.pool4(enc4_2)
        #print("enc4_1 size = ", enc4_1.shape)
        #print("enc4_2 size = ", enc4_2.shape)
        #print("pool4 size = ", pool4.shape)

        enc5_1 = self.enc5_1(pool4)
        #print("enc5_1 size = ", enc5_1.shape)

        # decoder part
        dec5_1 = self.dec5_1(enc5_1)
        #print("dec5_1 size = ", dec5_1.shape)

        unpool4 = self.unpool4(dec5_1)
        #print("unpool4 size = ", unpool4.shape)

        cat4 = self.copy_and_crop(enc4_2, unpool4)
        dec4_2 = self.dec4_2(cat4)
        dec4_1 = self.dec4_1(dec4_2)
        #print("cat4 size = ", cat4.shape)
        #print("dec4_2 size = ", dec4_2.shape)
        #print("dec4_1 size = ", dec4_1.shape)

        unpool3 = self.unpool3(dec4_1)
        #print("unpool3 size = ", unpool3.shape)

        cat3 = self.copy_and_crop(enc3_2, unpool3)
        dec3_2 = self.dec3_2(cat3)
        dec3_1 = self.dec3_1(dec3_2)
        #print("cat3 size = ", cat3.shape)
        #print("dec3_2 size = ", dec3_2.shape)
        #print("dec3_1 size = ", dec3_1.shape)

        unpool2 = self.unpool2(dec3_1)
        #print("unpool2 size = ", unpool2.shape)

        cat2 = self.copy_and_crop(enc2_2, unpool2)
        dec2_2 = self.dec2_2(cat2)
        dec2_1 = self.dec2_1(dec2_2)
        #print("cat2 size = ", cat2.shape)
        #print("dec2_2 size = ", dec2_2.shape)
        #print("dec2_1 size = ", dec2_1.shape)

        unpool1 = self.unpool1(dec2_1)
        #print("unpool1 size = ", unpool1.shape)

        cat1 = self.copy_and_crop(enc1_2, unpool1)
        dec1_2 = self.dec1_2(cat1)
        dec1_1 = self.dec1_1(dec1_2)
        #print("cat1 size = ", cat1.shape)
        #print("dec1_2 size = ", dec1_2.shape)
        #print("dec1_1 size = ", dec1_1.shape)

        x = self.conv(dec1_1)
        #print("Output size = ", x.shape)
        output_width, output_height = x.shape[2], x.shape[3]

        """
        [mirroring extrapolation]

        input(572*572)/output(388*388) 사이즈가 다르므로 output에 
        mirroring extrapolation 기법으로 missing context 부분을 채운다.
        """
        pad_width = int((input_width - output_width) / 2)
        pad_height = int((input_height - output_height) / 2)
        extrapolation = []
        for image in x:
            #image = nn.functional.pad(image, (pad_width, pad_width, pad_height, pad_height), 'reflect') # 이미지, (좌, 우, 상, 하), 방법
            image = nn.functional.pad(image, (pad_width, pad_width, pad_height, pad_height), 'constant', 0)
            extrapolation.append(image)

        output = torch.stack(extrapolation, dim=0).to(DEVICE)
        #print("Mirrored padding output size = ", output.shape)
        #print("Constant padding output size = ", output.shape)

        return output

#### 3-1-3) Dice score (구현중)

In [ ]:
class DiceLoss(nn.Module):
    def __init__(self):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        
        inputs = F.sigmoid(inputs) # sigmoid를 통과한 출력이면 주석처리
        
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()                            
        dice = (2.*intersection + smooth) / (inputs.sum() + targets.sum() + smooth)  
        
        return 1 - dice 

### 3-2) Pytorch

#### 3-2-1) Dataset

In [ ]:
class ScarDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform

        lst_data = os.listdir(self.data_dir)

        lst_scar = [f for f in lst_data if f.startswith('scar')]
        lst_label = [f for f in lst_data if f.startswith('label')]

        lst_scar.sort()
        lst_label.sort()

        self.lst_scar = lst_scar
        self.lst_label = lst_label

    def __len__(self):
        return len(self.lst_label)

    def __getitem__(self, index):
        scar = cv2.imread(os.path.join(self.data_dir, self.lst_scar[index]),
                          cv2.IMREAD_GRAYSCALE)
        label = cv2.imread(os.path.join(self.data_dir, self.lst_label[index]),
                          cv2.IMREAD_GRAYSCALE)
        #scar = np.load(os.path.join(self.data_dir, self.lst_scar[index]))
        #label = np.load(os.path.join(self.data_dir, self.lst_label[index]))

        scar = scar/255.0
        label = label/255.0

        if scar.ndim == 2:
            scar = scar[:, :, np.newaxis]
        if label.ndim == 2:
            label = label[:, :, np.newaxis]

        data = {'scar': scar, 'label': label}

        if self.transform:
            data = self.transform(data)

        return data

#### 3-2-2) Transform

In [ ]:
class ToTensor(object):
    def __call__(self, data):
        scar, label = data['scar'], data['label']
        scar = scar.transpose((2, 0, 1)).astype(np.float32)
        label = label.transpose((2, 0, 1)).astype(np.float32)

        data = {'scar': torch.from_numpy(scar), 'label': torch.from_numpy(label)}

        return data

# Dataset arugmentation
class Normalization(object):
    def __init__(self, mean=0.5, std=0.5):
        self.mean = mean
        self.std = std

    def __call__(self, data):
        scar, label = data['scar'], data['label']

        scar = (scar - self.mean) / self.std
        data = {'scar': scar, 'label': label}

        return data

class RandomFlip(object):
    def __call__(self, data):
        scar, label = data['scar'], data['label']

        if np.random.rand() > 0.5:
            scar = np.fliplr(scar)
            label = np.fliplr(label)

        if np.random.rand() > 0.5:
            scar = np.flipud(scar)
            label = np.flipud(label)

        data = {'scar': scar, 'label': label}

        return data

## *4) Run*

### 4-1) Prepare dataset

In [ ]:
trans = transforms.Compose([Normalization(),
                            RandomFlip(),
                            ToTensor()])

train_dataset = ScarDataset(data_dir=MODEL_PATH + DATASET_PATH + TRAIN_PATH, transform=trans)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

val_dataset = ScarDataset(data_dir=MODEL_PATH + DATASET_PATH + VAL_PATH, transform=trans)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)

### 4-2) Set variable about training

In [ ]:
net = UNet().to(DEVICE)
#criterion = nn.BCEWithLogitsLoss().to(DEVICE)
criterion = DiceLoss().to(DEVICE)

# Set optimizer
opt = torch.optim.Adam(net.parameters(), lr=LR)

num_batch_train = len(train_dataset) / BATCH_SIZE
num_batch_val = len(val_dataset) / BATCH_SIZE

# Set lambda func
fn_tonumpy = lambda x: x.to('cpu').detach().numpy().transpose(0, 2, 3, 1)
fn_denorm = lambda x, mean, std: (x * std) + mean
fn_class = lambda x: 1.0 * (x > 0.5)

# Set SummaryWriter
if not os.path.exists(MODEL_PATH + LOG_PATH + TRAIN_PATH):
    os.makedirs(MODEL_PATH + LOG_PATH + TRAIN_PATH)

if not os.path.exists(MODEL_PATH + LOG_PATH + VAL_PATH):
    os.makedirs(MODEL_PATH + LOG_PATH + VAL_PATH)

writer_train = SummaryWriter(log_dir=MODEL_PATH + LOG_PATH + TRAIN_PATH)
writer_val = SummaryWriter(log_dir=MODEL_PATH + LOG_PATH + VAL_PATH)

### 4-3) Train model

In [ ]:
# Load saved model
net, opt, start_epoch = load(MODEL_PATH + CHECK_POINT_PATH, net, opt)

# Init epoch
start_epoch = 0

for epoch in range(start_epoch+1, EPOCHS+1):
    net.train()
    loss_arr = []

    for batch, data in enumerate(train_loader, 1):
        # Forward
        scar = data['scar'].to(DEVICE)
        label = data['label'].to(DEVICE)
        output = net(scar)

        # Backward
        opt.zero_grad()

        loss = criterion(output, label)
        loss.backward()

        opt.step()

        # Save loss
        loss_arr += [loss.item()]
        print("==================================================")
        print(f"TRAIN || EPOCH {epoch :04d} | BATCH {batch : 04d} / {math.ceil(len(train_dataset)/BATCH_SIZE) : 04d} | LOSS {np.mean(loss_arr) : .4f}")
        print("==================================================")

        # Save tensorboard
        label = fn_tonumpy(label)
        scar = fn_tonumpy(fn_denorm(scar, mean=0.5, std=0.5))
        output = fn_tonumpy(fn_class(output))

        writer_train.add_image('label', label, num_batch_train * (epoch - 1) + batch, dataformats='NHWC')
        writer_train.add_image('scar', scar, num_batch_train * (epoch - 1) + batch, dataformats='NHWC')
        writer_train.add_image('output', output, num_batch_train * (epoch - 1) + batch, dataformats='NHWC')

    writer_train.add_scalar('loss', np.mean(loss_arr), epoch)

    # Validate
    with torch.no_grad():
        net.eval()
        loss_arr = []

        for batch, data in enumerate(val_loader, 1):
            # Forward
            scar = data['scar'].to(DEVICE)
            label = data['label'].to(DEVICE)
            output = net(scar)

            # Calc loss
            loss = criterion(output, label)

            # Save loss
            loss_arr += [loss.item()]
            print("==================================================")
            print(f"VALID || EPOCH {epoch :04d} | BATCH {batch : 04d} / {math.ceil(len(val_dataset)/BATCH_SIZE) : 04d} | LOSS {np.mean(loss_arr) : .4f}")
            print("==================================================")

            # Save tensorboard
            label = fn_tonumpy(label)
            scar = fn_tonumpy(fn_denorm(scar, mean=0.5, std=0.5))
            output = fn_tonumpy(fn_class(output))

            writer_val.add_image('label', label, num_batch_train * (epoch - 1) + batch, dataformats='NHWC')
            writer_val.add_image('scar', scar, num_batch_train * (epoch - 1) + batch, dataformats='NHWC')
            writer_val.add_image('output', output, num_batch_train * (epoch - 1) + batch, dataformats='NHWC')

        writer_val.add_scalar('loss', np.mean(loss_arr), epoch)

    # Save network at epoch end
    save(ckpt_dir=ckpt_dir, net = net, optim = optim, epoch = epoch)
    print("ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ")

writer_train.close()
writer_val.close()

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


TRAIN || EPOCH 0001 | BATCH  001 /  073 | LOSS  0.9895
TRAIN || EPOCH 0001 | BATCH  002 /  073 | LOSS  0.9887
TRAIN || EPOCH 0001 | BATCH  003 /  073 | LOSS  0.9846
TRAIN || EPOCH 0001 | BATCH  004 /  073 | LOSS  0.9831
TRAIN || EPOCH 0001 | BATCH  005 /  073 | LOSS  0.9816
TRAIN || EPOCH 0001 | BATCH  006 /  073 | LOSS  0.9823
TRAIN || EPOCH 0001 | BATCH  007 /  073 | LOSS  0.9834
TRAIN || EPOCH 0001 | BATCH  008 /  073 | LOSS  0.9827
TRAIN || EPOCH 0001 | BATCH  009 /  073 | LOSS  0.9834
TRAIN || EPOCH 0001 | BATCH  010 /  073 | LOSS  0.9830
TRAIN || EPOCH 0001 | BATCH  011 /  073 | LOSS  0.9836
TRAIN || EPOCH 0001 | BATCH  012 /  073 | LOSS  0.9829
TRAIN || EPOCH 0001 | BATCH  013 /  073 | LOSS  0.9835
TRAIN || EPOCH 0001 | BATCH  014 /  073 | LOSS  0.9838
TRAIN || EPOCH 0001 | BATCH  015 /  073 | LOSS  0.9835
TRAIN || EPOCH 0001 | BATCH  016 /  073 | LOSS  0.9839
TRAIN || EPOCH 0001 | BATCH  017 /  073 | LOSS  0.9839
TRAIN || EPOCH 0001 | BATCH  018 /  073 | LOSS  0.9841
TRAIN || E

KeyboardInterrupt: ignored

### 4-4) Show train result

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir {MODEL_PATH + LOG_PATH}